In [1]:
%cd ../
%matplotlib inline

/home/eli/AnacondaProjects/HTFATorch


In [2]:
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils

import csv
import glob
import logging
import os
import re

import nibabel as nib
from nilearn.input_data import NiftiMasker
import numpy as np
import scipy.io as sio

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

db_file = 'data/hollywood_mashup.db'
mask_file = '/data/datasets/siskind/fmri-datasets/hollywood-mashup/processed/experiments/subject-01/full-mask.nii.gz'

OVERWRITE = True

In [4]:
dataset = sio.loadmat('/data/datasets/siskind/fmri-datasets/hollywood-mashup/processed/experiments/subject-01/verb-from-video/dataset-3.mat')

In [5]:
mask = NiftiMasker(mask_strategy='background', smoothing_fwhm=None, standardize=False)
mask.fit(mask_file)

NiftiMasker(detrend=False, dtype=None, high_pass=None, low_pass=None,
      mask_args=None, mask_img=None, mask_strategy='background',
      memory=Memory(location=None), memory_level=1, sample_mask=None,
      sessions=None, smoothing_fwhm=None, standardize=False, t_r=None,
      target_affine=None, target_shape=None, verbose=0)

In [6]:
template = nib.load('/data/datasets/siskind/fmri-datasets/hollywood-mashup/processed/experiments/subject-01/epi.nii.gz')
sform = template.get_sform()

In [7]:
voxel_coordinates = np.array(np.nonzero(mask.mask_img_.dataobj)).transpose()
voxel_coordinates = np.hstack((voxel_coordinates,
                               np.ones((voxel_coordinates.shape[0], 1))))
voxel_locations = (voxel_coordinates @ sform.T)[:, :3]

In [8]:
def collect_run(run):
    index = dataset['chunks'][0, :] == run
    activations = dataset['data'][index]
    stimuli = dataset['targets'][index]
    return activations, stimuli

In [9]:
runs = [collect_run(i) for i in range(0, 48)]

In [10]:
def nii_run(activations):
    coords = np.round(np.array(
        np.dot(voxel_locations - sform[:3, 3],
               np.linalg.inv(sform[0:3, 0:3]))
    )).astype(int)
    data = np.zeros(template.shape[0:3] + (activations.shape[0],))
    
    for i in range(activations.shape[0]):
        for j in range(voxel_locations.shape[0]):
            x, y, z = coords[j, 0], coords[j, 1], coords[j, 2]
            data[x, y, z, i] = activations[i, j]
    
    return nib.Nifti1Image(data, affine=sform)

In [11]:
def save_run(activations, i):
    run = nii_run(activations)
    filename = '/data/datasets/siskind/fmri-datasets/hollywood-mashup/processed/experiments/subject-01/subject-01_run-%02d.nii.gz' % (i+1)
    nib.save(run, filename)
    return filename

In [12]:
existed = os.path.isfile(db_file)
db = niidb.FMriActivationsDb(db_file, mask=mask_file, smooth=6)

In [13]:
if not existed or OVERWRITE:
    for (i, (activations, stimuli)) in enumerate(runs):
        filename = save_run(activations, i)
        
        for (t, stimulus) in enumerate(stimuli):
            collected_block = niidb.FMriActivationBlock(zscore=False)
            collected_block.block = i * 72 + t
            collected_block.filename = filename
            collected_block.run = i
            collected_block.subject = 0
            collected_block.task = stimulus
            
            collected_block.start_time = t
            collected_block.end_time = t + 1
            
            db.upsert(collected_block)
            logging.info('Subject observing verb %s in run %d', stimulus, i)
            logging.info('Block %d belongs to run archive %s', collected_block.block, filename)

07/22/2019 14:27:49 Subject observing verb Eat         in run 0
07/22/2019 14:27:49 Block 0 belongs to run archive /data/datasets/siskind/fmri-datasets/hollywood-mashup/processed/experiments/subject-01/subject-01_run-01.nii.gz
07/22/2019 14:27:49 Subject observing verb AnswerPhone in run 0
07/22/2019 14:27:49 Block 1 belongs to run archive /data/datasets/siskind/fmri-datasets/hollywood-mashup/processed/experiments/subject-01/subject-01_run-01.nii.gz
07/22/2019 14:27:49 Subject observing verb SitDown     in run 0
07/22/2019 14:27:49 Block 2 belongs to run archive /data/datasets/siskind/fmri-datasets/hollywood-mashup/processed/experiments/subject-01/subject-01_run-01.nii.gz
07/22/2019 14:27:50 Subject observing verb GetOutCar   in run 0
07/22/2019 14:27:50 Block 3 belongs to run archive /data/datasets/siskind/fmri-datasets/hollywood-mashup/processed/experiments/subject-01/subject-01_run-01.nii.gz
07/22/2019 14:27:50 Subject observing verb HandShake   in run 0
07/22/2019 14:27:50 Block 4 

In [14]:
logging.info('Finished building NiiDb out of Hollywood-mashup dataset')

07/22/2019 14:33:11 Finished building NiiDb out of Hollywood-mashup dataset
